In [1]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
    repo_id="QuantFactory/Meta-Llama-3-8B-Instruct-GGUF",
    filename="*.Q8_0.gguf",
    n_gpu_layers=-1,
    verbose=True,
    n_ctx=2048,
    chat_format="chatml"
)
# llm.chat_format = "chatml"

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/abdo/.cache/huggingface/hub/models--QuantFactory--Meta-Llama-3-8B-Instruct-GGUF/snapshots/071f0830e6b83051d4626c5d95c1b33ebcbedad0/./Meta-Llama-3-8B-Instruct.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:          

In [4]:
import os

os.path.exists("/opt/conda/envs/llm")

True

In [1]:
from llama_cpp import Llama
from llama_cpp.llama_chat_format import Llava16ChatHandler

chat_handler = Llava16ChatHandler.from_pretrained(
  repo_id="cjpais/llava-v1.6-34B-gguf",
  filename="*mmproj*",
)

llava = Llama.from_pretrained(
  repo_id="cjpais/llava-v1.6-34B-gguf",
  filename="*.Q8_0.gguf",
  chat_handler=chat_handler,
  n_gpu_layers=-1,
  n_ctx=2048, # n_ctx should be increased to accomodate the image embedding
)

clip_model_load: loaded meta data with 25 key-value pairs and 378 tensors from /home/abdo/.cache/huggingface/hub/models--cjpais--llava-v1.6-34B-gguf/snapshots/98ab830fff4feade759748c831db5e1186c67e31/./mmproj-model-f16.gguf
clip_model_load: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
clip_model_load: - kv   0:                       general.architecture str              = clip
clip_model_load: - kv   1:                      clip.has_text_encoder bool             = false
clip_model_load: - kv   2:                    clip.has_vision_encoder bool             = true
clip_model_load: - kv   3:                   clip.has_llava_projector bool             = true
clip_model_load: - kv   4:                          general.file_type u32              = 1
clip_model_load: - kv   5:                               general.name str              = vit-large336-custom
clip_model_load: - kv   6:                        general.description str              = image encoder f

In [ ]:
def chat(ocr_pages: dict):
    responses = {}
    for pagenum, extracted_text in ocr_pages.items():
        response = llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": "You are a server API that receives receipt information and returns its items and prices as a JSON object.",
                },
                {"role": "user", "content": f"{extracted_text}"},
            ],
            response_format={
                "type": "json_object",
            },
            temperature=0.7,
        )
        responses[pagenum] = response
    return responses

In [ ]:
import base64
import io
from PIL import Image

def encode_image(image_path):
    buffer = io.BytesIO()
    image = Image.open(image_path)
    image.resize((128, 128))
    image.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode('utf-8')

In [12]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [13]:
image_path = "27786.jpg"
base64_image = encode_image(image_path)

In [3]:
response = llm.create_chat_completion(
            messages=[
              {
                    "role": "system",
                    "content": "You are an assistant that receives a receipt image and returns the required information as a JSON object.",
                },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": " Mountain Mountain\n Heritage Heritage\n HOTEL·BLUEMOUNTAINS·AUSTRALIA HOTEL·BLUEMOUNTAINS·AUSTRALIA HOTEL·BLUEMOUNTAINS·AUSTRALIA\n Mountain Heritage Mountain Heritage\n Restaurant\n 6/10ApexSt\n KatoombaNSW2780\n Australia\n Phone:（02)47822155\n ABN:15654364851\n GUESTBILL\n BEVERAGE\n Vodka Martini 20.00\n CokeNoSugar300ml CokeNoSugar300ml 6.00\n BEVERAGESubtotal: BEVERAGESubtotal: $26.00\n FOOD\n Scallops 15.00\n Quail 15.00\n Beef Tenderloin 15.00\n SpannerCrab 15.00\n 2Course\n 2@$45.00 2@$45.00 90.00\n MAINAWAY 0.00\n FOOD Subtotal: $150.00\n AMOUNTDUE: $176.00\n GST total in sale: $16.00\n Receipt#:16205 Receipt#:16205\n Date:17/12/2022Time:7:36:32PM Date:17/12/2022Time:7:36:32PM\n Clerk:***MH\n Table:403\n Terminal:402MtHeritageP0S2 Terminal:402MtHeritageP0S2"
            },
          {
            "type": "text",
            "text": """This is a receipt. Extract the following information for each item: name and price (quantity if available) in a JSON format.
            **Look for lines containing item descriptions. Extract the text before the price as the item name. Extract the number next to the item name (if present) as the quantity. Extract the numbers after '$' as the price."""
          },
          # {
          #   "type": "image_url",
          #   "image_url": {
          #     "url": f"data:image/jpeg;base64,{base64_image}"
          #   }
          # }
        ]
      }
    ],
            response_format={
                "type": "json_object",
            }
        )

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\<U+0000>-<U+001F>] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE

In [ ]:
import json
response["choices"][0]["message"]["content"]

In [ ]:
responses = chat(
    {
        "page_1": ' Mountain Mountain\n Heritage Heritage\n HOTEL·BLUEMOUNTAINS·AUSTRALIA HOTEL·BLUEMOUNTAINS·AUSTRALIA HOTEL·BLUEMOUNTAINS·AUSTRALIA\n Mountain Heritage Mountain Heritage\n Restaurant\n 6/10ApexSt\n KatoombaNSW2780\n Australia\n Phone:（02)47822155\n ABN:15654364851\n GUESTBILL\n BEVERAGE\n Vodka Martini 20.00\n CokeNoSugar300ml CokeNoSugar300ml 6.00\n BEVERAGESubtotal: BEVERAGESubtotal: $26.00\n FOOD\n Scallops 15.00\n Quail 15.00\n Beef Tenderloin 15.00\n SpannerCrab 15.00\n 2Course\n 2@$45.00 2@$45.00 90.00\n MAINAWAY 0.00\n FOOD Subtotal: $150.00\n AMOUNTDUE: $176.00\n GST total in sale: $16.00\n Receipt#:16205 Receipt#:16205\n Date:17/12/2022Time:7:36:32PM Date:17/12/2022Time:7:36:32PM\n Clerk:***MH\n Table:403\n Terminal:402MtHeritageP0S2 Terminal:402MtHeritageP0S2',
    }
)

In [ ]:
for page, response in responses.items():
    print(page, response)

In [ ]:
responses["page_1"]["choices"][0]["message"]["content"]